## TL;DR

Missing Data (NA) in Pandas:

- Representation
  - `None` object
  - `np.nan` floating-point
  - In Pandas both are nearly interchangeable
- Operation on NA
  - Detection
    - `isnull()`
    - `notnull()`
  - Drop
    - `dropna()`
      - Default: drop all rows containing NA
      - Drop along a specified axis (via `axis` parameter)
      - Fine control:
        - `how` parameter
        - `thresh` parameter
  - Fill
    - With single value
    - forward/backward fill

In [2]:
import numpy as np
import pandas as pd

Data in the real world is that real-world data is rarely clean and homogeneous.

## Trade-offs in Missing Data Conventions

Strategies to indicate the presence of missing data:

### Using a mask that globally indicates missing values

- How: \
the mask might be an entirely separate Boolean array, or it may involve appropriation of one bit in the data representation to locally indicate the null status of a value.

- Trade-off:\
use of a separate mask array requires allocation of an additional Boolean array, which adds overhead in both storage and computation. 

### Choosing a sentinel value that indicates a missing entry

- How: \
The sentinel value could be some data-specific convention, such as 
    - indicating a missing integer value with -9999 or some rare bit pattern, or 
    - it could be a more global convention, such as indicating a missing floating-point value with NaN (Not a Number), 
    - a special value which is part of the IEEE floating-point specification.
    
    
- Trade-off:\
A sentinel value reduces the range of valid values that can be represented, and may require extra (often non-optimized) logic in CPU and GPU arithmetic. Common special values like NaN are not available for all data types.

## Missing Data in Pandas

- Special floating-point `NaN` value

- Python `None` object

### `None`: Pythonic missing data

`None`, a python singleton object that is often used for missing data in python code. 

Because it is a Python object, None cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type `object`.

In [3]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` across an array with a `None` value, you will generally get an error:

In [4]:
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### `NaN`: Missing numerical data 

`NaN`: 

- acronym for "Not a Number"

- is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation. And there's no equivalent NaN value for integers, strings, or other types.

In [5]:
vals2= np.array([1, np.nan, 3, 4])
vals2

array([ 1., nan,  3.,  4.])

In [6]:
vals2.dtype

dtype('float64')

`NaN`  is a bit like a data virus. It infects any other object it touches:

The result of arithmetic with `NaM` will be another `NaN`

In [7]:
1 + np.nan

nan

In [8]:
0 * np.nan

nan

In [9]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

Numpy prodives some special aggregations that will ignore these missing values:

In [11]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### `NaN` and `None` in Pandas

Panda is built to handle `NaN` and `None` nearly **interchangeably**, converting between them where appropriate

In [12]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.

In [13]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [18]:
x[0] = None

As `np.nan` is a floating-point, `x` will automatically be upcast to a floating-point type to accommodate the NA

In [17]:
x

0    NaN
1    1.0
dtype: float64

The following table lists the upcasting conventions in Pandas when NA values are introduced:

| Typeclass  | Conversion When Storing NAs | NA Sentinel Value  |
| :--------- | :-------------------------- | :----------------- |
| `floating` | No change                   | `np.nan`           |
| `object`   | No change                   | `None` or `np.nan` |
| `integer`  | Cast to `float64`           | `np.nan`           |
| `boolean`  | Cast to `object`            | `None` or `np.nan` |

Keep in mind that in Pandas, string data is always stored with an `object` dtype.

## Operating on Null Values

Methods for detecting, removing, and replacing null values in Pandas data structures:

- `isnull()`: Generate a boolean mask indicating missing values
- `notnull()`: Opposite of `isnull()`
- `dropna()`: Return a filtered version of the data
- `fillna()`: Return a copy of the data with missing values filled or imputed

### Detecting null values

Two useful methods for detecting null data: `isnull()` and `notnull()`. Either one will return a Boolean mask over the data.

In [20]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [22]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [23]:
data.notnull()

0     True
1    False
2     True
3    False
dtype: bool

 Boolean masks can be used directly as a `Series` or `DataFrame` index

In [24]:
data[data.isnull()]

1     NaN
3    None
dtype: object

In [25]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

- `dropna()`: removes NA values

- `fillna()`: fills in NA values

#### `Series`

In [26]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [27]:
data.dropna()

0        1
2    hello
dtype: object

In [28]:
data.fillna(0)

0        1
1        0
2    hello
3        0
dtype: object

#### `DataFrame`

In [29]:
df = pd.DataFrame([[1, np.nan, 2], 
                   [2, 3, 5], 
                   [np.nan, 4, 6]])

In [30]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


For `DataFrame`, `dropna()`: 

- drop all rows containing null value by default

In [31]:
df.dropna()

,0,1,2
1,2.0,3.0,5


- can drop NA values along a different axis

In [32]:
# Drop all columns containing a null value
df.dropna(axis=1)

,2
0,2
1,5
2,6


In [33]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


- Use `how` or `thresh` parameters, which allo fine control of the number of nulls to walk through

    - Default is `how=any`: any row or column (depending on the `axis` keyword) containing a null value will be dropped
    
    - `how=all`: only drop rows/columns that are all null values
    
    - 'thresh': specify a minimum number of non-null values for the row/column to be kept

In [34]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [37]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [38]:
# Drop the columns which contain Na(s)
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [40]:
# Drop the columns which are all NAs
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [41]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [39]:
# drop the rows which do NOT contain at least 3 non-NA values
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


The first and last row have been dropped, because they contain only two non-null values.

### Filling null values

In [42]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
FILLdata

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

Fill NA entries with a single value:

In [43]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

Use forward-fill to propagate the previous value forward:

In [44]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

Use back-fill to propagate the next values backward:

In [45]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For `DataFrame`, options are similar, but we can specify an `axis` along which the fills take place:

In [46]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [47]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [50]:
df.fillna(method='bfill', axis=0)

,0,1,2,3
0,1.0,3.0,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


Notice: if a previous value is not available during a forward/backward fill, the NA value remains.